# Set up and global variables

In [ ]:
import os

from pathlib import Path
from collections import defaultdict
from copy import deepcopy

import matplotlib.pyplot as plt
import seaborn as sns
import pandas as pd
import numpy as np
import random
import json

from matplotlib.colors import ListedColormap
from IPython.display import display, HTML
from tqdm import tqdm
from scipy.stats import spearmanr

import src.ipython_loader as loader

from src.prioritization import *
from src.utils import split_data_by_users, gini

RESOLUTION = 300
VERSION = '0.0.0'
DATASET_PATH = Path('data') / 'datasets' / f'ipython_{VERSION}'
OUTPUT_PATH = DATASET_PATH / 'heuristics'
BINARY_CMAP = ListedColormap(['red', 'green'])

IMAGE_DIR = Path('images') / "heuristics"

CACHE_PATH = Path('data') / 'cache'

os.makedirs(CACHE_PATH, exist_ok=True)
os.makedirs(OUTPUT_PATH, exist_ok=True)
os.makedirs(IMAGE_DIR, exist_ok=True)

MIN_TASK_DEFECT_SUBMISSIONS = 5

***

# Loading data

In [ ]:
items = pd.read_csv(DATASET_PATH / f'items_{VERSION}.csv', index_col=0)
log = pd.read_csv(DATASET_PATH / f'log_{VERSION}.csv', index_col=0, parse_dates=['time'])
defects = pd.read_csv(DATASET_PATH / f'defects_{VERSION}.csv', index_col=0)
defect_log = pd.read_csv(DATASET_PATH / f'defect_log_{VERSION}.csv', index_col=0)
defect_log.columns = defect_log.columns.astype(int)
code_to_defect_id = json.load(open(DATASET_PATH / f'code_to_defect_id_{VERSION}.json', "r"))
defect_presence = defect_log > 0

***
# Data Split

In [ ]:
train_log, test_log, train_defect_log, test_defect_log = split_data_by_users(log, defect_log)

In [ ]:
test_defects = test_defect_log > 0
print("Fraction of Submissions with Multiple Defects:", (test_defects.sum(axis=1) > 1).mean())
print("Total Number", (test_defects.sum(axis=1) > 1).sum())

***

# Heuristics

In [ ]:
data = items, defects

models = {
    "Task Common": TaskCommonModel(*data),
    "Task Characteristic": TaskCharacteristicModel(*data),
    "Student Frequency": StudentFrequencyModel(*data),
    "Student Characteristic": StudentCharacteristicModel(*data),
    "Student Encountered": StudentEncounteredBeforeModel(*data),
    "Defect Multiplicity": DefectMultiplicityModel(*data),
    "Severity Baseline": SeverityModel(*data),
}

In [ ]:
for name, model in (pbar :=tqdm(models.items(), desc="Training Models")):
    pbar.set_description(f"Training {name}")
    model.update(train_log, train_defect_log)

***

# Pilot testing

## function definitions

In [ ]:
# Written with ChatGPT

def _get_prioritized_defects(submission, defect_counts, active_models: dict, defects: pd.DataFrame) -> pd.DataFrame:
    """Generate a DataFrame of prioritized defects for a single submission.

    Run all active models on a single submission and return a DataFrame of 
    defects ranked by consensus score, including the original scores.
    """
    present_defects = defect_counts[defect_counts > 0]
    all_defect_scores = pd.DataFrame(index=present_defects.index)
    
    if present_defects.empty:
        return all_defect_scores

    for name, model in active_models.items():
        scores = model.prioritize(submission, defect_counts)
        # Store the priority score for display
        all_defect_scores[name] = scores.loc[present_defects.index]
    
    # Calculate rank for sorting (sum of ranks across all models for visualization)
    all_defect_scores['SUM_RANK'] = all_defect_scores.rank(ascending=False).sum(axis=1)
    
    # Sort by consensus rank (lower sum rank = higher consensus priority)
    sorted_defects_df = all_defect_scores.sort_values(by='SUM_RANK', ascending=True)
    return sorted_defects_df

def _generate_submission_html(sub_num: int, submission: pd.Series, sorted_defects_df: pd.DataFrame, active_models: dict, items: pd.DataFrame, defects: pd.DataFrame) -> str:
    """
    Generate HTML description of a single submission.
    
    This includes the submission number, task info, submitted code, and the defects table with
    colored prioritization scores.
    """
    task_id = submission['item']
    task_row = items.loc[task_id]
    present_defects_count = len(sorted_defects_df)
    
    html_output = f"""
    <style>
        .defect-tooltip {{
            position: relative;
            display: inline-block;
            cursor: help;
            border-bottom: 1px dashed #fff; /* Use light dash for dark background */
        }}
        .defect-tooltip .tooltiptext {{
            visibility: hidden;
            width: 400px;
            background-color: #1e1e1e; /* Deeper dark background */
            color: #f5f5f5; /* Light text color */
            text-align: left;
            border-radius: 6px;
            padding: 10px;
            position: absolute;
            z-index: 1000;
            /* FIX 4: Adjusted tooltip position to right/center */
            top: 50%;
            left: 100%; 
            transform: translateY(-50%) translateX(10px); /* Center vertically, move right 10px */
            opacity: 0;
            transition: opacity 0.3s;
            font-family: monospace;
            white-space: normal;
            box-shadow: 0 4px 8px rgba(0,0,0,0.5);
        }}
        .defect-tooltip:hover .tooltiptext {{
            visibility: visible;
            opacity: 1;
        }}
        .tooltiptext strong {{
            color: #FFD700;
        }}
        .tooltiptext pre {{
            background-color: #333; /* Darker pre background */
            padding: 5px;
            border-radius: 3px;
            overflow-x: auto;
        }}
        /* FIX 2: Dark Mode for Instructions Panel (Increased Contrast) */
        .instructions-panel {{
            background-color: #333 !important; /* Darker background */
            color: #f5f5f5 !important; /* Light text color */
            border: 1px solid #555 !important;
        }}
        /* FIX 3: Dark Mode for Table Headers (Increased Contrast) */
        .defect-table-header {{
            background-color: #444 !important; /* Clearly dark background */
            color: #f5f5f5 !important; /* Light text color */
        }}
        /* FIX 5: Dark Mode for Table Body Cells */
        .data-cell {{
            border: 1px solid #555;
            padding: 8px;
            text-align: center;
            background-color: #333; /* Dark background */
            color: #f5f5f5; /* Light text */
        }}
        .data-cell-left {{
            border: 1px solid #555;
            padding: 8px;
            text-align: left;
            background-color: #333;
            color: #f5f5f5;
            font-weight: bold;
        }}
    </style>
    <div style="border: 2px solid #555; margin: 20px 0; padding: 15px; border-radius: 8px; background-color: #222; color: #f5f5f5;">
        <div style="background-color: #4CAF50; color: white; padding: 8px; border-radius: 5px; margin-bottom: 15px;">
            <h3 style="margin: 0;">SUBMISSION {sub_num + 1} | Task ID: {task_id} ({task_row['display name']})</h3>
            <p style="margin: 0; font-size: 0.9em;">Submitted at: {submission['time']}</p>
        </div>
        
        <!-- Task and Submission Content -->
        <div style="display: flex; gap: 15px; margin-bottom: 15px;">
            <div class="instructions-panel" style="flex: 1; padding: 10px; border-radius: 5px;">
                <strong>Instructions:</strong>
                <p style="font-size: 0.9em; max-height: 100px; overflow-y: auto;">{task_row['instructions']}</p>
            </div>
            <div style="flex: 1.5; border: 1px solid #ddd; padding: 10px; border-radius: 5px; background-color: #2e2e2e; color: #f5f5f5; font-family: monospace;">
                <strong>Submitted Code:</strong>
                <pre style="margin: 0; max-height: 150px; overflow-y: auto; white-space: pre-wrap;">{submission['answer']}</pre>
            </div>
        </div>

        <!-- Defect Prioritization Table -->
        <h4 style="margin-top: 20px;">Prioritized Defects (Defects Present: {present_defects_count})</h4>
        <table style="width: 100%; border-collapse: collapse; font-size: 0.9em;">
            <thead>
                <tr class="defect-table-header">
                    <th style="border: 1px solid #555; padding: 8px; width: 30%; text-align: left;">Defect (Hover for Details)</th>
    """
    
    # Add a column header for each model
    for name in active_models.keys():
        html_output += f"<th style=\"border: 1px solid #555; padding: 8px; width: 12%;\">{name}<br>({active_models[name].get_measure_name()})</th>"
    
    html_output += "</tr></thead><tbody>"
    
    # --- 5. Generate Table Rows ---
    for defect_id in sorted_defects_df.index:
        defect_row = defects.loc[defect_id]        
        # --- Tooltip Content Generation (Sanitized HTML) ---
        
        # Start with required content
        tooltip_content = f"""
        <strong>Type:</strong> {defect_row['defect type']}<br>
        <strong>Description:</strong> {defect_row['description']}<br>
        """
        
        # Helper to conditionally add code blocks
        def add_code_block(title, key, content):
            if pd.isna(content) or content == "":
                return ""
            escaped_content = content.replace('<', '&lt;').replace('>', '&gt;')
            return f"""
            <strong>{title}:</strong> <pre>{escaped_content}</pre>
            """

        # Conditionally add Example Code
        tooltip_content += add_code_block(
            "Example Code", 
            'code example', 
            defect_row['code example']
        )

        # Conditionally add Suggested Fix
        tooltip_content += add_code_block(
            "Suggested Fix", 
            'code fix example', 
            defect_row['code fix example']
        )
        
        html_output += "<tr>"
        
        # Defect Name with Tooltip
        html_output += f"""
        <td class="data-cell-left">
            <div class="defect-tooltip">{defect_row['display name']}
                <span class="tooltiptext">{tooltip_content}</span>
            </div>
        </td>
        """        
        # Model Scores
        for name in active_models.keys():
            score = sorted_defects_df.loc[defect_id, name]
            
            # --- Score Visualization (Color Highlighting) ---
            min_score = sorted_defects_df[name].min()
            max_score = sorted_defects_df[name].max()
            
            if max_score == min_score:
                normalized_score = 0.5 # Neutral color if all scores are identical
            else:
                # Map score from [min, max] to a visual intensity range [0.3, 1.0]
                normalized_score = np.interp(score, [min_score, max_score], [0.3, 1.0])
            
            # Create a reddish hue where intensity reflects score
            r = 255
            g = b = 255 - int(200 * normalized_score)
            g = max(0, g)
            b = max(0, b)
            
            color = f"rgb({r}, {g}, {b})"

            html_output += f"<td class=\"data-cell\" style=\"background-color: {color};\">{score:.3f}</td>"
        
        html_output += "</tr>"
    
    html_output += "</tbody></table>"
    html_output += "</div>"
    return html_output


def pilot_test_user(user_id: int, models: dict, log: pd.DataFrame, defect_log: pd.DataFrame, items: pd.DataFrame, defects: pd.DataFrame):
    """
    Analyze a single user's submission history chronologically.

    Args:
        user_id: The ID of the student to analyze.
        models: Dictionary of model instances to test.
        log: Full log DataFrame (for user filtering).
        defect_log: Full defect matrix.
        items: Full items/tasks DataFrame.
        defects: Full defects DataFrame.
    """
    # --- 1. Prepare User Data ---
    user_log = log[log['user'] == user_id].sort_values('time')
    user_defect_log = defect_log.loc[user_log.index]
    
    if user_log.empty:
        display(HTML(f"<h2>No submissions found for User ID: {user_id}</h2>"))
        return
    
    print(f"--- STARTING PILOT TEST FOR USER: {user_id} ---")

    # create temporary models to avoid modifying the original
    temp_models = deepcopy(models)
    
    # --- 2. Iterate Through Submissions Chronologically ---
    for sub_num, (index, submission) in enumerate(tqdm(user_log.iterrows(), total=len(user_log), desc=f"User {user_id} Submissions")):
        defect_counts = user_defect_log.loc[index]
        
        # --- Prioritization and Ranking ---
        sorted_defects_df = _get_prioritized_defects(submission, defect_counts, temp_models, defects)
        
        # --- HTML Generation and Display ---
        html_output = _generate_submission_html(
            sub_num, submission, sorted_defects_df, temp_models, items, defects
        )
        display(HTML(html_output))
            
        # --- 3. Update Models ---
        # The update is called on the single Series submission and defect_counts
        for model in temp_models.values():
            model.update(submission, defect_counts)
            
    print(f"--- PILOT TEST COMPLETE FOR USER: {user_id} ---")


## output

In [ ]:
pilot_test_user(test_log.iloc[110]['user'], models, test_log, test_defect_log, items, defects)